## Урок 3. Парсинг HTML

1) Необходимо собрать информацию о вакансиях на должность программиста или разработчика с сайта superjob.ru или hh.ru. (Можно с обоих сразу) Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе: 
- Наименование вакансии, 
- Предлагаемую зарплату,
- Ссылку на саму вакансию

In [165]:
from bs4 import BeautifulSoup as bs
import requests
import re

def get_link(main_link, add_link, key_word):
    link = main_link + add_link + key_word
    return link

def get_html(link):
    if link[0:4] == 'http':
        html = requests.get(link).text
    else:
        with open(link, 'rb') as f:
            html = f.read()
    return html

def parsing_html(link):
    html =  get_html(link)
    parsed_html = bs(html, 'html.parser')
    return parsed_html

def find_class(html, class_):
    found_class = html.findAll(class_=class_)
    return found_class

def find_item(html, class_, href=False):
    found_item = html.find(class_=class_, href=href)
    return found_item

def get_vacancy(html, class_):
    found_class = find_class(html, class_[0])
    class_info = []
    for i in found_class:
        class_dict = {}
        vacancy = find_item(i, class_[1], href=True)
        class_dict['vacancy'] = vacancy.get_text()
        salary = find_item(i, class_[2])
        class_dict['salary'] = ''.join(re.findall('[0-9-]', salary.get_text())) if salary else '-'
        link = vacancy['href']
        class_dict['link'] = link if link[0:4] == 'http' else main_link + link
        class_info.append(class_dict)
    return class_info

#### hh.ru - НЕ работает GET запрос через requests, поэтому заранее записан HTML-файл в браузере.

In [166]:
main_link = 'https://hh.ru'
add_link = '/search/vacancy'
key_word = '?text=Программист Разработчик'
class_ = ['vacancy-serp-item', 'bloko-link HH-LinkModifier', 'vacancy-serp-item__compensation']

# link = get_link(main_link, add_link, key_word)
link = 'hh.html'
parsed_html = parsing_html_page(link)
vacancy_hh = get_vacancy(parsed_html, class_)
vacancy_hh

[{'vacancy': 'Веб-программист',
  'salary': '180000-200000',
  'link': 'https://hh.ru/vacancy/32979955?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA'},
 {'vacancy': 'Веб-программист',
  'salary': '180000-200000',
  'link': 'https://hh.ru/vacancy/32979956?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA'},
 {'vacancy': 'Golang Developer',
  'salary': '4000-6000',
  'link': 'https://hh.ru/vacancy/33127912?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA'},
 {'vacancy': 'Web-программист',
  'salary': '90000-120000',
  'link': 'https://hh.ru/vacancy/32895066?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA'},
 {'vacancy':

#### superjob.ru - работает GET запрос через requests, поэтому данные получаем напрямую с сайта

In [167]:
main_link = 'https://superjob.ru'
add_link = '/vacancy/search/'
key_word = '?keywords=Программист|Разработчик'
class_ = ['_3syPg', '_1QIBo', '_2Wp8I']

link = get_link(main_link, add_link, key_word)
# link = 'sj.html'
parsed_html = parsing_html_page(link)
vacancy_sj = get_vacancy(parsed_html, class_)
vacancy_sj

[{'vacancy': 'Программист-разработчик 1С',
  'salary': '120000',
  'link': 'https://superjob.ru/vakansii/programmist-razrabotchik-1s-32066066.html'},
 {'vacancy': 'Программист / Разработчик',
  'salary': '80000',
  'link': 'https://superjob.ru/vakansii/programmist-32274027.html'},
 {'vacancy': 'Разработчик 1С (Программист)',
  'salary': '170000',
  'link': 'https://superjob.ru/vakansii/razrabotchik-1s-32191974.html'},
 {'vacancy': 'Web-программист, Web-разработчик',
  'salary': '',
  'link': 'https://superjob.ru/vakansii/web-programmist-32266222.html'},
 {'vacancy': 'Программист / Разработчик 1С 8.3 УПП',
  'salary': '140000',
  'link': 'https://superjob.ru/vakansii/programmist-32418578.html'},
 {'vacancy': 'Программист-разработчик (.NET) на ELMA BPM',
  'salary': '',
  'link': 'https://superjob.ru/vakansii/programmist-razrabotchik-32408682.html'},
 {'vacancy': 'Разработчик / Программист (Битрикс)',
  'salary': '',
  'link': 'https://superjob.ru/vakansii/razrabotchik-32409339.html'},
 

2) Доработать приложение таким образом, чтобы можно было искать разработчиков на разные языки программирования (Например Python, Java, C++)